In [3]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import time as time


from bcg_code import *

from IPython.display import clear_output

# lanczos time

In [38]:
n = 10

np.geomspace(1,100,n),\
100**((np.arange(n))/(n-1))

(array([  1.        ,   1.66810054,   2.7825594 ,   4.64158883,
          7.74263683,  12.91549665,  21.5443469 ,  35.93813664,
         59.94842503, 100.        ]),
 array([  1.        ,   1.66810054,   2.7825594 ,   4.64158883,
          7.74263683,  12.91549665,  21.5443469 ,  35.93813664,
         59.94842503, 100.        ]))

In [40]:
experiments = []

# intro
# n = 4000
# Λ = np.geomspace(1,100,n)

# experiments.append({
#     'name': 'intro',
#     'Λ': Λ,
#     'blocksizes': [1,5,20,100,200],
#     'qs': [650,200,60,11,7],
#     'skips': [20,4,1,1,1],
#     'ks': [50,100,200]
# })

# duplicate eigenvalues
n = 4000
Λ = np.hstack([np.geomspace(1,100,n//2),np.geomspace(1,100,n//2)])

experiments.append({
    'name': 'doubles',
    'Λ': Λ,
    'blocksizes': [1,5,20,100,200],
    'qs': [650,200,60,11,7],
    'skips': [20,4,1,1,1],
    'ks': [50,100,200]
})

# TW example
# n = 4000
# Λ = np.exp(-(1+np.arange(n))/25)

# experiments.append({
#     'name': 'fastdecay',
#     'Λ': Λ,
#     'blocksizes': [1,5,20,100,200],
#     'qs': [220,60,20,5,4],
#     'skips': [10,2,1,1,1],
#     'ks': [50,100,200],
#     'ylims': [1e-6,1e1],
# })

# n = 4000
# Λ = np.max([np.exp(-(1+np.arange(n))/25),(1-(1+np.arange(n))/(n+1))/25],axis=0)

# experiments.append({
#     'name': 'slowdecay',
#     'Λ': Λ,
#     'blocksizes': [1,5,20,100,200],
#     'qs': [400,180,50,13,10],
#     'skips': [20,4,1,1,1],
#     'ks': [50,100,200],
#     'ylims': [1e-6,1e1],
# })

In [41]:
for experiment in experiments:

    # load experiment
    name = experiment['name']
    Λ = experiment['Λ']
    blocksizes = experiment['blocksizes']
    qs = experiment['qs']
    skips = experiment['skips']
    ks = experiment['ks']

    A = np.diag(Λ)
    class AAt:
        def __matmul__(self,X):
            return A@(A.T@X)
            
    AAt = AAt()

    # run experiment
    all_times = {}
    all_errs = {}
    for i,(b,q,skip) in enumerate(zip(blocksizes,qs,skips)):
    
        B = np.random.randn(n,b)
        
        Q,_,times = block_lanczos(AAt,B,q,reorth=True)
    
        all_times[b] = times
    
        for k in ks:
            all_errs[b,k] = np.zeros(q+1)
            
        for j in range(0,q+1,skip):
        
            Zj = Q[:,:b*j]
    
            X = Zj.T@A
            Qtilde,s,Vt = np.linalg.svd(X,full_matrices=False)
    
            for k in ks:
                print(i,j,k)
                clear_output(wait=True)
                
                A_hat = Zj@(Qtilde[:,:k]@(np.diag(s[:k])@Vt[:k]))
                err = np.linalg.norm(A - A_hat)
                
                all_errs[b,k][j] = err

    np.save(f'data/{name}',[all_errs,all_times])

4 7 200
